# Extract all textual data from associated link pages using keyword search
# 이 코드는 키워드를 입력하면 
1) 1차로 구글검색을 통해서 결과룰 추출 - 연결링크 모두 수집 <br>
2) 2차로 링크페이지에 모두 접속하여 text 데이터를 추출하여 리스트로 돌려주는(출력) 기능

In [1]:
# Chrome 버전을 확인하고 드라이버 버전을 동일하게 해야 함
# 크롬드라이버 다운로드 링크 : https://chromedriver.chromium.org/downloads
# 현재 사용하는 크롬 버전 90.0.4430.85(공식 빌드) (x86_64)
# 적용한 크롬 드라이버 버전은 위와 동일

In [2]:
import re
import pandas as pd
import openpyxl
from urllib.parse import ParseResultBytes, quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver

from tqdm import tqdm

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words = set(stopwords.words('english')) 

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# 혹은 options.add_argument("--disable-gpu")

# UserAgent값을 바꿔줍시다! 서버가 인식하지 못하도록 가상으로 headless 값 추가함ㅠ
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")


In [3]:
def cleaning_data(input_data):
    remove = input_data.replace("  "," ") # 변환
    remove_ = re.sub(r"\t", " ", remove) # 제거
    remove__ = re.sub(r"\n", " ", remove_) # 제거
    remove__ = remove__.replace("   ", " ")
    remove__ = remove__.replace("  ", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace("…/", " ")
    remove__ = remove__.replace("…", " ")
    remove__ = remove__.replace("/", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace(")", ",")
    remove__ = remove__.replace("(", ",")
    preprossed = remove__.split(",") # 단어를 리스트로 변환
    #print(preprossed)
    
    # 표제어 추출, 동사는 현재형으로 변환
    lemma_list =[]
    for i in preprossed:
        lema_re = lemmatizer.lemmatize(i, pos='v') #표제어 추출, 동사는 현재형으로 변환
        lemma_list.append(lema_re)
    
    # 표제어 추출
    ext_lema = [lemmatizer.lemmatize(w) for w in preprossed]
    # 중복값을 제거하고
    rm_dupli = set(ext_lema)
    # 다시 리스트로 만들고
    re_li = list(rm_dupli)
    # 빈 값은 제거하고
    get_wd =list(filter(None, re_li))
    # 소문자로 모두 변환
    lower_wd = [i.lower() for i in get_wd]
    
    result = []
    for w in lower_wd: 
        if w not in stop_words: 
            result.append(w)
    return result

In [4]:
def google_search_result_tp_knowledge(input_word):
    baseUrl = 'https://www.google.com/search?q='

    #plusUrl = input('무엇을 검색할까요? :')
    plusUrl = input_word

    # url = baseUrl + quote_plus(plusUrl)
    url = baseUrl + plusUrl
    # 한글을 사용할 경우 :  quote_plus 적용 - URL에 막 %CE%GD%EC 이런 거 생성해줌

    driver = webdriver.Chrome(executable_path= r'./data/chromedriver_mac_ver_90', chrome_options=options)
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, features="html.parser")

    v = soup.select('.yuRUbf')

    search_title_result = []
    search_linked_contents_result = []
    for i in v:
        #print(i.select_one('.LC20lb.DKV0Md').text)
        search_title_result.append(i.select_one('.LC20lb.DKV0Md').text)
        print(i.a.attrs['href'])
        search_linked_contents_result.append(i.a.attrs['href'])
        #print()


    # search_linked_contents_result 의 각 링크로 접속하여 해당 내용을 모두 text로 크롤링한 후, body내의 단어들만 추출한다.
    get_all_linked_web_data = []
    for linked_page in tqdm(search_linked_contents_result):
        driver.get(linked_page)
        html = driver.page_source
        get_all_data = BeautifulSoup(html, features="html.parser")
        get_all_linked_web_data.append(get_all_data)
        
    body = re.search('<body.*/body>', html, re.I|re.S)
    if (body is None):
        print ("No <body> in html")
        exit()
            
    body = body.group()
    #print(body)
    
    # 추출된 정보 클린징
    korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')#한글제거

    item_extract = str(body).replace('\n', ' ')
    item_extract = re.sub('<span.*?>.*?</span>', ' ', item_extract)
    item_extract = re.sub('<b>.*?</b>', ' ', item_extract)    
    item_extract = re.sub('<.*?>', ' ', item_extract)        
    item_extract = item_extract.replace('\t', ' ')
    item_extract = re.sub(korean, '', item_extract)
    item_extract = re.sub('[-=.#/?:$}]', ' ', item_extract)
    item_extract = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』;{}()'\\‘|\(\)\[\]\<\>`\'…》]", ' ', item_extract)
    #print (item_extract)
    
    driver.close()

    get_result_str = str(item_extract)
    result_cln = cleaning_data(get_result_str) # 결과값 청소
    result = list(set(result_cln)) # 중복제거

    return result

In [5]:
# run ##

#result = google_search_result(input_word)
result = google_search_result_tp_knowledge("college personal essay")

print(result)


/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()
  0%|          | 0/13 [00:00<?, ?it/s]

https://www.collegeessayguy.com/blog/college-essay-examples
https://www.princetonreview.com/college-advice/college-essay
https://www.collegebasics.com/applying-to-college/seven-ways-to-make-your-college-essay-stand-out/
https://blog.collegevine.com/8-questions-your-college-essay-should-answer/
https://www.fastweb.com/college-search/articles/college-admissions-essay-topics-to-avoid
https://www.shemmassianconsulting.com/blog/college-essay-examples
https://www.usnews.com/education/best-colleges/articles/2019-04-24/college-essay-examples-how-to-write-your-story
https://www.usnews.com/education/best-colleges/articles/how-to-write-a-college-essay
https://www.princetonreview.com/college-advice/college-essay
https://blog.prepscholar.com/what-is-a-personal-statement-everything-you-need-to-know-about-the-college-essay
https://blog.prepscholar.com/college-essay-examples-that-worked-expert-analysis
https://apply.jhu.edu/application-process/essays-that-worked/
https://blog.collegevine.com/personal-

100%|██████████| 13/13 [00:48<00:00,  3.74s/it]


['real', 'grade', 'urging', 'might', 'new', 'applicant', '2023', '2021', 'texas', 'worried', 'unfair', 'like', 'bag', 'html', 'reading', 'year', 'communities', 'journey', 'http', 'subscribe', 'sat', 'graduate', 'felt', 'surprised', 'enabled', 'spice', 'questions', 'course', 'written', 'becoming', 'feed', 'students’', 'write', 'community', 'window', 'road', 'personal', 'drove', 'application', 'name', 'document', 'flagname', 'ce8524c1e3a3', 'seen', 'careers', 'thousand', 'exercise', 'understand', 'find', 'middle', 'addition', 'play', 'strong', 'early', '2416790', 'freedom', 'j', 'b2iexixdbaepgrxfeiepirt1wzaib2dmwjtx5va2', 'fifth', 'posts', 'cardemail', 'confirmation', 'converter', 'true', 'rest', 'what’s', 'way', 'terms', '4c28', 'test', 'financial', 'youssef', 'post', 'let', 'change', 'difference', 'applying', 'bus', 'lock', 'you’ll', 'interrupted', 'newsletter', 'mistake', 'around', 'pageuri', 'review', 'cs', 'stellar', 'information', 'officer', '5', 'ground', '6d41c6e5', 'earlier', 'a